# Exploratory Data Analysis

In [85]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

## Initial insight

In [88]:
df = pd.read_csv('bet_table.csv')
df.head()

,Stakes,Odds,Available,Num_Runners,Race_Type,Going,Days,Bet_Strength,Result
0,2,2.90,41,6,FLAT,GOOD/SOFT,33,2,LOSER
1,2,7.20,1589,10,HURDLE,FIRM,19,3,WINNER
2,2,1.04,169,14,HURDLE,HEAVY,6,2,WINNER
3,2,2.26,75,9,FLAT,FAST,15,2,LOSER
4,5,1.83,248,16,HURDLE,SOFT,14,4,LOSER


In [108]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3208 entries, 0 to 3207
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Stakes        3208 non-null   int64  
 1   Odds          3208 non-null   float64
 2   Available     3208 non-null   int64  
 3   Num_Runners   3208 non-null   int64  
 4   Race_Type     3208 non-null   object 
 5   Going         3208 non-null   object 
 6   Days          3208 non-null   int64  
 7   Bet_Strength  3208 non-null   int64  
 8   Result        3208 non-null   object 
 9   Profit        3208 non-null   float64
dtypes: float64(2), int64(5), object(3)
memory usage: 250.8+ KB


,Stakes,Odds,Available,Num_Runners,Days,Bet_Strength,Profit
count,3208.000000,3208.000000,3208.000000,3208.000000,3208.000000,3208.000000,3208.000000
mean,2.914589,29.212291,346.107544,10.910536,27.662406,2.765898,1.619330
std,1.381263,110.710028,467.496642,4.483811,19.151875,1.359028,12.902352
min,2.000000,1.010000,2.000000,5.000000,5.000000,1.000000,-5.000000
25%,2.000000,1.607500,45.000000,7.000000,15.000000,2.000000,-2.000000
50%,2.000000,2.470000,139.500000,10.000000,21.000000,3.000000,-2.000000
75%,5.000000,5.600000,437.500000,13.000000,35.000000,4.000000,3.840000
max,5.000000,900.000000,2150.000000,26.000000,125.000000,5.000000,600.000000


### Calculate profit row

In [109]:
def calculate_profit(row):
    if row['Result'] == 'WINNER':
        return row['Stakes'] * row['Odds']
    elif row['Result'] == 'LOSER':
        return -row['Stakes']
    else:
        return 0

df['Profit'] = df.apply(calculate_profit, axis=1)
df.head()

,Stakes,Odds,Available,Num_Runners,Race_Type,Going,Days,Bet_Strength,Result,Profit
0,2,2.90,41,6,FLAT,GOOD/SOFT,33,2,LOSER,-2.00
1,2,7.20,1589,10,HURDLE,FIRM,19,3,WINNER,14.40
2,2,1.04,169,14,HURDLE,HEAVY,6,2,WINNER,2.08
3,2,2.26,75,9,FLAT,FAST,15,2,LOSER,-2.00
4,5,1.83,248,16,HURDLE,SOFT,14,4,LOSER,-5.00


### Num_Runners vs Win_Rate

In [116]:
ave_profit_num_runners = df.groupby('Num_Runners').Profit.mean().reset_index()
win_rate_num_runners = df.Result.eq('WINNER').groupby(df.Num_Runners).mean().mul(100).reset_index()
num_runners_df = ave_profit_num_runners.merge(win_rate_num_runners, how='inner', on='Num_Runners')
num_runners_df.rename(columns = {
    'Profit': 'Mean_Profit',
    'Result': 'Win_Rate'
}, inplace=True)

In [149]:
x = list(num_runners_df.Num_Runners)
y = list(num_runners_df.Win_Rate)
z = np.polyfit(x, y, 2)
f = np.poly1d(z)
x_new = np.linspace(x[0], x[-1], 50)
y_new = f(x_new)  # Polyfitted x values

trace1 = go.Scatter(
    x=num_runners_df['Num_Runners'],
    y=num_runners_df['Win_Rate'],
    line = dict(color='crimson', width=4)
)

trace2 = go.Scatter(
    x=x_new,
    y=y_new,
    mode='lines',
    name='Fit',
    line = dict(color='crimson', width=2, dash='dash')
    )

layout = go.Layout(
    title='The effect of the number of racers on percentage of bets won.',
    xaxis_title='Number of runners',
    yaxis_title='Win rate (%)',
    showlegend=False
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.write_image('../images/num_racers.png')
fig